## Import cities_df

In [2]:
import pandas as pd

In [3]:
cities_df= pd.read_pickle("cities2.pkl")  

## Get Weather from API

In [4]:
import requests
import json
from api_key import API_key

In [5]:
def ConvertCoord(coord):
    #convert lat and long
    lat,long = coord.replace("°E","").replace("°N","").split(" ")
    if lat.find("°S") != -1 :
        lat="-" + lat.replace("°S","")
    if long.find("°W") != -1 :
        long="-" + long.replace("°W","")
    try:
        lat= float(lat)
        long= float(long)
    except:
        print("ERROR, COORDINATES are not Convertable")
    return (lat,long)

In [ ]:
# coor= "52.52000°N 13.40500°E"  
# lat,long = ConvertCoord(coor)
# print(lat,long)
# url_weather= f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={long}&appid={API_key}"
# weather_response = requests.get(url_weather)
# weather_json_l= weather_response.json()
# #print(json.dumps(weather_json_l, indent=2))

52.52 13.405


In [6]:
weather_json_l=[]
for coor in cities_df.coordinate:
    lat,long = ConvertCoord(coor)
    url_weather= f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={long}&appid={API_key}&units=metric"
    weather_response = requests.get(url_weather)
    if weather_response.status_code != 200:
        print(weather_response.text)
        weather_json = ""
    else:
        weather_json_l.append(weather_response.json())
       

In [ ]:
# from pprint import pprint
# pprint(weather_json_l[0])

In [7]:
cities_weath = {"cities":[]
                ,"forcast_time":[]
                ,"outlook":[]
                ,"temperature":[]
                ,"feels_like":[]
                ,"wind_speed":[]
                ,"rain_prob":[]
                }

for j,city_json in enumerate(weather_json_l):
    current_city = cities_df.loc[j].cities
    for time in city_json["list"]:
        cities_weath["cities"].append(current_city)
        cities_weath["forcast_time"].append(time["dt_txt"])
        cities_weath["temperature"].append(time["main"]["temp"])
        cities_weath["outlook"].append(time["weather"][0]["description"])
        cities_weath["feels_like"].append(time["main"]["feels_like"])
        cities_weath["wind_speed"].append(time["wind"]["speed"])
        cities_weath["rain_prob"].append(time["pop"])
        

In [8]:
cities_weather_df = pd.DataFrame(cities_weath)

### Final df

In [9]:
cities_weather_df

,cities,forcast_time,outlook,temperature,feels_like,wind_speed,rain_prob
0,London,2023-01-04 12:00:00,light rain,12.91,12.42,7.97,0.30
1,London,2023-01-04 15:00:00,overcast clouds,12.45,11.76,7.48,0.00
2,London,2023-01-04 18:00:00,overcast clouds,10.98,10.17,6.71,0.00
3,London,2023-01-04 21:00:00,overcast clouds,10.69,9.80,6.37,0.00
4,London,2023-01-05 00:00:00,overcast clouds,10.92,9.97,6.06,0.00
...,...,...,...,...,...,...,...
515,Vienna,2023-01-08 21:00:00,overcast clouds,5.70,4.89,1.39,0.00
516,Vienna,2023-01-09 00:00:00,light rain,6.37,5.47,1.53,0.36
517,Vienna,2023-01-09 03:00:00,moderate rain,5.76,2.27,5.03,1.00
518,Vienna,2023-01-09 06:00:00,moderate rain,5.24,2.02,4.24,1.00
